In [2]:
%matplotlib inline
from skimage import data, io, filters, exposure
from skimage.segmentation import slic, felzenszwalb
from skimage.transform import resize
from skimage.morphology import square, closing, binary_closing
from matplotlib import pyplot as plt
from skimage import color
import numpy as np
from scipy import ndimage as ndi
from skimage.measure import moments_hu, moments, moments_central, moments_normalized

In [3]:
def get_feature(name):
    sample = io.imread(str(name))
#     print(sample.shape)
    sample = resize(sample, (148,224))
#     plt.imshow(sample)
#     print(sample.shape)

    sample_hsv = color.convert_colorspace(sample,'RGB','HSV')
#     print(sample_hsv[90,125])
    for i in range(sample_hsv.shape[0]):
        for j in range(sample_hsv.shape[1]):
            if sample_hsv[i,j,0] > 0.2083 and sample_hsv[i,j,0] < 0.4167 and sample_hsv[i,j,2]>0.1  and sample_hsv[i,j,1]>0.1:
                sample_hsv[i,j,1] = 0
                sample_hsv[i,j,2] = 1
            else:
                sample_hsv[i,j,2] = 0
#     print(sample_hsv[90,125])
    sample_rgb = color.convert_colorspace(sample_hsv,'HSV','RGB')
#     print(sample_rgb[90,125])
    sample_grey = color.rgb2grey(sample_rgb)
#     print(sample_grey[90,125])
#     plt.imshow(sample_grey, cmap=plt.get_cmap('gray'))

    t = 0
    res_i = 0
    res_j = 0
    for i in range(sample_grey.shape[0]-100):
        for j in range(sample_grey.shape[1]-100):
            if np.sum(sample_grey[i:i+100,j:j+100]) > t:
                res_i = i
                res_j = j
                t = np.sum(sample_grey[i:i+100,j:j+100])
#     print(t, res_i, res_j)
    sample_cropped = sample_grey[res_i:res_i+100,res_j:res_j+100]
#     plt.imshow(sample_cropped, cmap=plt.get_cmap('gray'))

    closed = binary_closing(sample_cropped, square(2))
#     plt.imshow(closed, cmap=plt.get_cmap('gray'))
    closed = np.asarray(closed, dtype=np.double)

    M = moments(closed)
    cr = M[1, 0] / M[0, 0]
    cc = M[0, 1] / M[0, 0]
#     print(cr,cc)
    mu = moments_central(closed, cr=cr, cc=cc)
    nu = moments_normalized(mu)
    return moments_hu(nu)

In [4]:
feature = np.empty((0,8), int)

from os import listdir
from os.path import isfile, join
mypath = 'train_1'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for o in onlyfiles:
    feature = np.vstack((feature, np.hstack((get_feature(join(mypath,o)),0))))

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [5]:
mypath = 'train_2'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for o in onlyfiles:
    feature = np.vstack((feature,np.hstack((get_feature(join(mypath,o)),1))))

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [ ]:
feature_test = np.empty((0,8), int)
mypath = 'test'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

x = 0
for o in onlyfiles:
    x = int(o[8:10])
    if x<67:
        u=0
    else:
        u=1
    feature_test = np.vstack((feature_test,np.hstack((get_feature(join(mypath,o)),u))))
    x += 1
feature_test.shape

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(21, 8)

In [ ]:
from sklearn.neural_network import MLPClassifier
# 15 0.6 50
# 9-3, 200, 0.1, 1500
mlp = MLPClassifier(hidden_layer_sizes=(9,3),
                    solver='sgd',
                    batch_size=200,
                    activation='relu',
                    learning_rate='adaptive',
                    learning_rate_init=0.1,
                    max_iter=1500,
                    momentum=0,
                    tol=0.0,
                    alpha=0,
                    verbose=True,
                    random_state=123)
train_x = feature[:,:-1]
train_y = feature[:,-1]
test_x = feature_test[:,:-1]
test_y = feature_test[:,-1]
test_x.shape, test_y.shape

mlp.fit(train_x, train_y)
print(mlp.score(test_x,test_y))
print(mlp.predict(test_x))

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:358: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn("Got `batch_size` less than 1 or larger than "


Iteration 1, loss = 0.69875291
Iteration 2, loss = 0.69715169
Iteration 3, loss = 0.69575997
Iteration 4, loss = 0.69455219
Iteration 5, loss = 0.69350562
Iteration 6, loss = 0.69260004
Iteration 7, loss = 0.69181753
Iteration 8, loss = 0.69114221
Iteration 9, loss = 0.69056010
Iteration 10, loss = 0.69005886
Iteration 11, loss = 0.68962768
Iteration 12, loss = 0.68925705
Iteration 13, loss = 0.68893871
Iteration 14, loss = 0.68866541
Iteration 15, loss = 0.68843086
Iteration 16, loss = 0.68822961
Iteration 17, loss = 0.68805692
Iteration 18, loss = 0.68790870
Iteration 19, loss = 0.68778142
Iteration 20, loss = 0.68767203
Iteration 21, loss = 0.68757791
Iteration 22, loss = 0.68749682
Iteration 23, loss = 0.68742682
Iteration 24, loss = 0.68736627
Iteration 25, loss = 0.68731374
Iteration 26, loss = 0.68726804
Iteration 27, loss = 0.68722812
Iteration 28, loss = 0.68719312
Iteration 29, loss = 0.68716227
Iteration 30, loss = 0.68713496
Iteration 31, loss = 0.68711062
Iteration 32, los

Iteration 269, loss = 0.68457434
Iteration 270, loss = 0.68456129
Iteration 271, loss = 0.68454820
Iteration 272, loss = 0.68453509
Iteration 273, loss = 0.68452195
Iteration 274, loss = 0.68450879
Iteration 275, loss = 0.68449559
Iteration 276, loss = 0.68448237
Iteration 277, loss = 0.68446912
Iteration 278, loss = 0.68445584
Iteration 279, loss = 0.68444253
Iteration 280, loss = 0.68442919
Iteration 281, loss = 0.68441583
Iteration 282, loss = 0.68440243
Iteration 283, loss = 0.68438901
Iteration 284, loss = 0.68437555
Iteration 285, loss = 0.68436207
Iteration 286, loss = 0.68434856
Iteration 287, loss = 0.68433502
Iteration 288, loss = 0.68432145
Iteration 289, loss = 0.68430786
Iteration 290, loss = 0.68429423
Iteration 291, loss = 0.68428057
Iteration 292, loss = 0.68426688
Iteration 293, loss = 0.68425317
Iteration 294, loss = 0.68423942
Iteration 295, loss = 0.68422565
Iteration 296, loss = 0.68421184
Iteration 297, loss = 0.68419801
Iteration 298, loss = 0.68418414
Iteration 

Iteration 743, loss = 0.67387700
Iteration 744, loss = 0.67384226
Iteration 745, loss = 0.67380746
Iteration 746, loss = 0.67377261
Iteration 747, loss = 0.67373769
Iteration 748, loss = 0.67370273
Iteration 749, loss = 0.67366771
Iteration 750, loss = 0.67363263
Iteration 751, loss = 0.67359749
Iteration 752, loss = 0.67356230
Iteration 753, loss = 0.67352706
Iteration 754, loss = 0.67349176
Iteration 755, loss = 0.67345640
Iteration 756, loss = 0.67342099
Iteration 757, loss = 0.67338552
Iteration 758, loss = 0.67335000
Iteration 759, loss = 0.67331442
Iteration 760, loss = 0.67327879
Iteration 761, loss = 0.67324310
Iteration 762, loss = 0.67320736
Iteration 763, loss = 0.67317156
Iteration 764, loss = 0.67313571
Iteration 765, loss = 0.67309980
Iteration 766, loss = 0.67306384
Iteration 767, loss = 0.67302782
Iteration 768, loss = 0.67299175
Iteration 769, loss = 0.67295562
Iteration 770, loss = 0.67291944
Iteration 771, loss = 0.67288321
Iteration 772, loss = 0.67284692
Iteration 